In [1]:
import json
import os
import pickle
import re
from collections import Counter
from itertools import chain

import pandas as pd
from tqdm.notebook import tqdm

In [2]:
data = pd.read_csv("movie_bd_v5.csv")
data.sample(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
205,tt2011159,13200000,53830415,No Good Deed,Idris Elba|Taraji P. Henson|Leslie Bibb|Kate d...,Sam Miller,First he gets into your house. Then he gets in...,"Terri, a devoted wife and mother of two, lives...",84,Thriller,Sony Pictures|Screen Gems,9/12/2014,5.8,2014
1864,tt0203019,32000000,48814909,Men of Honor,Robert De Niro|Cuba Gooding Jr.|Charlize Thero...,"George Tillman, Jr.",History is made by those who break the rules.,Against formidable odds -- and an old-school d...,129,Drama,Fox 2000 Pictures,9/13/2000,7.0,2000
1569,tt0343818,120000000,347234916,"I, Robot",Will Smith|Bridget Moynahan|Alan Tudyk|James C...,Alex Proyas,Laws are made to be broken.,"In the year 2035, where robots are common and ...",115,Action|Science Fiction,Twentieth Century Fox Film Corporation|Laurenc...,7/15/2004,6.6,2004


In [3]:
data.dtypes

imdb_id                  object
budget                    int64
revenue                   int64
original_title           object
cast                     object
director                 object
tagline                  object
overview                 object
runtime                   int64
genres                   object
production_companies     object
release_date             object
vote_average            float64
release_year              int64
dtype: object

# Предобработка

In [4]:
answers = {}  # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

# the time given in the dataset is in string format.
# So we need to change this in datetime format
# ...

In [5]:
months_russian_names = {
    1: "Январь",
    2: "Февраль",
    3: "Март",
    4: "Апрель",
    5: "Май",
    6: "Июнь",
    7: "Июль",
    8: "Август",
    9: "Сентябрь",
    10: "Октябрь",
    11: "Ноябрь",
    12: "Декабрь",
}
seasons = {
    1: "winter",
    2: "winter",
    3: "spring",
    4: "spring",
    5: "spring",
    6: "summer",
    7: "summer",
    8: "summer",
    9: "autumn",
    10: "autumn",
    11: "autumn",
    12: "winter",
}

# трансформируем дату и создаём сезональные переменные
data["release_date"] = pd.to_datetime(data.release_date)
data["release_month"] = data.release_date.dt.month
data["release_day"] = data.release_date.dt.day
data["release_month_ru"] = data.release_month.map(months_russian_names)
data["season"] = data.release_month.map(seasons)

# создаём множества из переменных, в которых несколько значений
data["cast_set"] = data.cast.apply(lambda cast: set(cast.split("|")))
data["genres_set"] = data.genres.apply(lambda genres: set(genres.split("|")))
data["production_companies_set"] = data.production_companies.apply(
    lambda production_companies: set(production_companies.split("|"))
)

# переменные, связанные с количеством элементов
data["title_lengths"] = data.original_title.apply(lambda title: len(title))
data["overview_wordcounts"] = data.overview.apply(
    lambda overview: len(re.findall(r"\w+", overview))
)

# прочие переменные
data["profit"] = data.revenue - data.budget

data.sample(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,...,release_month,release_day,release_month_ru,season,cast_set,genres_set,production_companies_set,title_lengths,overview_wordcounts,profit
121,tt2911666,20000000,78739897,John Wick,Keanu Reeves|Michael Nyqvist|Alfie Allen|Wille...,Chad Stahelski|David Leitch,Don't set him off.,"After the sudden death of his beloved wife, Jo...",101,Action|Thriller,...,10,22,Октябрь,autumn,"{Alfie Allen, Willem Dafoe, Keanu Reeves, Mich...","{Action, Thriller}","{87Eleven, Thunder Road Pictures, DefyNite Fil...",9,81,58739897
580,tt0255798,22000000,84772742,The Animal,Rob Schneider|Colleen Haskell|John C. McGinley...,Luke Greenfield,He wasn't much of a man... Now he's not much o...,When loser Marvin Mange is involved in a horri...,84,Action|Comedy,...,6,1,Июнь,summer,"{Colleen Haskell, Guy Torry, Rob Schneider, Ed...","{Action, Comedy}","{Revolution Studios, Happy Madison Productions}",10,70,62772742
468,tt1341188,120000000,48668907,How Do You Know,Reese Witherspoon|Paul Rudd|Owen Wilson|Jack N...,James L. Brooks,How do you know it's love?,After being cut from the USA softball team and...,121,Comedy|Drama|Romance,...,12,17,Декабрь,winter,"{Reese Witherspoon, Paul Rudd, Owen Wilson, Ka...","{Romance, Comedy, Drama}","{Columbia Pictures, Road Rebel, Gracie Films}",15,42,-71331093


In [6]:
data.dtypes

imdb_id                             object
budget                               int64
revenue                              int64
original_title                      object
cast                                object
director                            object
tagline                             object
overview                            object
runtime                              int64
genres                              object
production_companies                object
release_date                datetime64[ns]
vote_average                       float64
release_year                         int64
release_month                        int64
release_day                          int64
release_month_ru                    object
season                              object
cast_set                            object
genres_set                          object
production_companies_set            object
title_lengths                        int64
overview_wordcounts                  int64
profit     

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.
Вы думаете и в жизни у вас будут варианты ответов?)

In [7]:
# тут пишем ваш код для решения данного вопроса:

In [8]:
answer_one = data.sort_values(["budget"], ascending=False).head(1)[
    ["original_title", "imdb_id"]
]

In [9]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример:
answers["1"] = "2. Spider-Man 3 (tt0413300)"
# запишите свой вариант ответа
answers["1"] = f"5. {answer_one.values[0][0]} ({answer_one.values[0][1]})"
# если ответили верно, можете добавить комментарий со значком "+"

In [10]:
answers["1"]
# +

'5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

In [11]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [12]:
title, id = data.loc[
    data.runtime == data.runtime.max(), ["original_title", "imdb_id"]
].values[0]
title, id

('Gods and Generals', 'tt0279111')

In [13]:

# думаю логику работы с этим словарем вы уже поняли,
# по этому не буду больше его дублировать
answers["2"] = f"2. {title} ({id})"

In [14]:
answers["2"]
# +

'2. Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [15]:
title, id = data.loc[
    data.runtime == data.runtime.min(), ["original_title", "imdb_id"]
].values[0]
title, id

('Winnie the Pooh', 'tt1449283')

In [16]:
answers["3"] = f"3. {title} ({id})"
answers["3"]
# +

'3. Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [17]:
answers["4"] = f"2. {data.runtime.mean():.0f}"
answers["4"]
# +

'2. 110'

# 5. Каково медианное значение длительности фильмов?

In [18]:
answers["5"] = f"1. {data.runtime.median():.0f}"
answers["5"]
# +

'1. 107'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget)

In [19]:
# лучше код получения столбца profit вынести в Предобработку что в начале
title, id = data.loc[
    data.profit == data.profit.max(), ["original_title", "imdb_id"]
].values[0]
title, id

('Avatar', 'tt0499549')

In [20]:
answers["6"] = f"5. {title} ({id})"
answers["6"]
# +

'5. Avatar (tt0499549)'

# 7. Какой фильм самый убыточный?

In [21]:
title, id = data.loc[
    data.profit == data.profit.min(), ["original_title", "imdb_id"]
].values[0]
title, id

('The Lone Ranger', 'tt1210819')

In [22]:
answers["7"] = f"5. {title} ({id})"
answers["7"]
# +

'5. The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [23]:
answers["8"] = f"1. {data.loc[data.profit > 0]['imdb_id'].count()}"
answers["8"]
# +

'1. 1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [24]:
title, id = (
    data.query("release_year == 2008")
        .sort_values(["revenue"])
        .tail(1)[["original_title", "imdb_id"]]
        .values[0]
)
title, id

('The Dark Knight', 'tt0468569')

In [25]:
answers["9"] = f"4. {title} ({id})"
answers["9"]
# +

'4. The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [26]:
title, id = (
    data.query("release_year >= 2012 & release_year <= 2014")
        .sort_values(["profit"])
        .head(1)[["original_title", "imdb_id"]]
        .values[0]
)
title, id

('The Lone Ranger', 'tt1210819')

In [27]:
answers["10"] = f"5. {title} ({id})"
answers["10"]
# +

'5. The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [28]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

In [29]:
most_frequent_genres = Counter(
    chain(*[g.split("|") for g in data.genres.tolist()])
).most_common()
most_frequent_genres

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

ВАРИАНТ 2

In [30]:
Counter(chain(*data.genres.apply(lambda genres: genres.split("|")))).most_common()

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

In [31]:
answers["11"] = f"3. {most_frequent_genres[0][0]}"
answers["11"]
# +

'3. Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными?

Я интерпретирую это так: "какой жанр встречается чаще среди фильмов с профитом выше нуля"

In [32]:
genres_list = [g[0] for g in most_frequent_genres]
genres_list[:3]

['Drama', 'Comedy', 'Thriller']

In [33]:
data.shape

(1889, 24)

In [34]:
data_profitable = data.query("profit > 0")
data_profitable.shape

(1478, 24)

In [35]:
genre_counts = {}
for genre in genres_list:
    genre_counts[genre] = data_profitable.loc[
        data_profitable.genres.str.contains(genre)
    ].shape[0]

genre_counts

{'Drama': 560,
 'Comedy': 551,
 'Thriller': 446,
 'Action': 444,
 'Adventure': 337,
 'Crime': 231,
 'Romance': 242,
 'Family': 226,
 'Science Fiction': 195,
 'Fantasy': 188,
 'Horror': 150,
 'Mystery': 119,
 'Animation': 120,
 'Music': 47,
 'History': 46,
 'War': 41,
 'Western': 12,
 'Documentary': 7,
 'Foreign': 0}

In [36]:
answers[
    "12"
] = f"1. {[k for k, v in genre_counts.items() if v == max(genre_counts.values())][0]}"
answers["12"]
# +

'1. Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [37]:
answers[
    "13"
] = f"5. {data.groupby('director')['revenue'].sum().sort_values().tail(1).index[0]}"
answers["13"]
# +

'5. Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

## для задания

In [38]:
directors_x_action_films = {}


def get_films_for_director(director):
    return data.loc[
        (data.director == director) & (data.genres.str.contains("Action"))
        ].shape[0]


for director in data.director.unique():
    directors_x_action_films[director] = get_films_for_director(director)

# Так сложилось, что система предлагает в качестве вариантов не самых-самых
# Не проблема, давайте глянем, сколько фильмов у предложенных вариантов
directors_for_puzzle = {
    k: v
    for k, v in directors_x_action_films.items()
    if k
       in [
           "Ridley Scott",
           "Guy Ritchie",
           "Robert Rodriguez",
           "Quentin Tarantino",
           "Tony Scott",
       ]
}
directors_for_puzzle

{'Ridley Scott': 6,
 'Quentin Tarantino': 3,
 'Guy Ritchie': 4,
 'Tony Scott': 5,
 'Robert Rodriguez': 6}

## на самом деле

(к сожалению, режиссёры с самым большим количестом фильмов жанра экшн
не фигурируют в вариантах ответа)

In [39]:
# Вот правильный ответ
# На случай, если вам интересно
{
    k: v
    for k, v in directors_x_action_films.items()
    if v == max(directors_x_action_films.values())
}

{'Michael Bay': 7, 'Paul W.S. Anderson': 7}

## Ладно, пишем ответ

Выберем лучшего из предложенных и удовлетворим систему

In [40]:
answers["14"] = f"3. {sorted([k for k, v in directors_for_puzzle.items() if v == max(directors_for_puzzle.values())])[-1]}"
answers["14"]
# -

'3. Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году?

In [41]:
actors = set(chain(*data.cast.str.split("|")))

In [42]:
most_revenuable_actors = {}

for actor in actors:
    most_revenuable_actors[actor] = data.loc[
        (data.release_year == 2012) & (data.cast.str.contains(actor))
        ].revenue.sum()

answers[
    "15"
] = f"3. {[k for k, v in most_revenuable_actors.items() if v == max(most_revenuable_actors.values())][0]}"
answers["15"]
# +

'3. Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [43]:
data_high_budget = data.loc[data.budget > data.budget.mean()]
data_high_budget.shape

(671, 24)

In [44]:
highcost_films_per_actor = {}

for actor in actors:
    highcost_films_per_actor[actor] = data_high_budget.loc[
        data_high_budget.cast.str.contains(actor)
    ].imdb_id.count()

answers[
    "16"
] = f"3. {[k for k, v in highcost_films_per_actor.items() if v == max(highcost_films_per_actor.values())][0]}"
answers["16"]
# +

'3. Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage?

In [45]:
answers[
    "17"
] = f"2. {Counter(chain(*data.loc[data.cast.str.contains('Nicolas Cage')].genres.str.split('|'))).most_common()[0][0]}"
answers["17"]
# +

'2. Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [46]:
# наверное пора бы уже написать функцию для этого типового решения...
# но не сегодня

# Правда, проще написать однострочник и отформатировать его в pep8.

title, id = (
    data.loc[data.production_companies.str.contains("Paramount Pictures")]
        .sort_values(["profit"])[["original_title", "imdb_id"]]
        .iloc[0, :]
        .values
)

answers["18"] = f"1. {title} ({id})"
answers["18"]
# +

'1. K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [47]:
# Больше однострочников Богу Однострочников!
answers[
    "19"
] = f"5. {data.groupby('release_year').revenue.sum().sort_values().tail(1).index[0]}"
answers["19"]
# +

'5. 2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [48]:
answers[
    "20"
] = f"1. {data.loc[data.production_companies.str.contains('Warner Bros')].groupby('release_year').profit.sum().sort_values().tail(1).index[0]}"
answers["20"]
# +

'1. 2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [49]:
answers[
    "21"
] = f'4. {data.groupby("release_month_ru").imdb_id.count().sort_values().tail(1).index[0]}'
answers["21"]
# +

'4. Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [50]:
answers["22"] = f'2. {data.groupby("season").imdb_id.count()["summer"]}'
answers["22"]
# +

'2. 450'

# 23. Для какого режиссера зима – самое продуктивное время года?

In [51]:
directors_x_seasons = data.pivot_table(
    index="director", columns="season", values="imdb_id", aggfunc="count"
).sort_values(["winter"], ascending=False)
directors_x_seasons

season,autumn,spring,summer,winter
director,,,,
Peter Jackson,1.0,NaN,NaN,7.0
Steven Soderbergh,3.0,2.0,2.0,6.0
Clint Eastwood,4.0,NaN,2.0,6.0
Adam Shankman,NaN,2.0,2.0,4.0
Martin Scorsese,2.0,NaN,NaN,4.0
...,...,...,...,...
Wong Kar-wai,NaN,1.0,NaN,NaN
Woody Allen,NaN,1.0,2.0,NaN
Xavier Gens,1.0,NaN,NaN,NaN


In [52]:
answers["23"] = f"5. {directors_x_seasons.head(1).index[0]}"
answers["23"]

'5. Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [53]:
data.groupby("production_companies").title_lengths.mean().sort_values()

production_companies
Focus Films|Starz Animation|Relativity Media|Tim Burton Productions|Arc Productions                                      1.0
Ixtlan Productions|Emperor Motion Pictures|Global Entertainment Group                                                    2.0
Columbia Pictures|Relativity Media                                                                                       2.0
Twentieth Century Fox Film Corporation|Donners' Company|Bad Hat Harry Productions|Marvel Enterprises|XM2 Productions     2.0
Sunswept Entertainment|Warner Bros.|Metro-Goldwyn-Mayer (MGM)                                                            3.0
                                                                                                                        ... 
Dune Entertainment|Fox 2000 Pictures|Walden Media                                                                       56.0
Plan B Entertainment|Virtual Studios|Scott Free Productions|Alberta Film Entertainment|Warner Bros.     

In [54]:
production_companies = set(
    chain(*data.production_companies.apply(lambda corp: corp.split("|")))
)
len(production_companies)

1771

In [55]:
corps_by_filme_title_length = {}

for corp in production_companies:
    corps_by_filme_title_length[corp] = data.loc[
        data.production_companies.str.contains(corp)
    ].title_lengths.mean()

# глянем, что там у нас в максимумах
{
    k: v
    for k, v in corps_by_filme_title_length.items()
    if v == max(corps_by_filme_title_length.values())
}

/home/user/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


{'Four By Two Productions': 83.0}

In [56]:
answers[
    "24"
] = f"5. {[k for k, v in corps_by_filme_title_length.items() if v == max(corps_by_filme_title_length.values())][0]}"
answers["24"]

'5. Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [57]:
corps_by_overview_wordcount = {}

for corp in production_companies:
    corps_by_overview_wordcount[corp] = data.loc[
        data.production_companies.str.contains(corp)
    ].overview_wordcounts.mean()

# глянем, что у нас попало в максимумы
{
    k: v
    for k, v in corps_by_overview_wordcount.items()
    if v == max(corps_by_overview_wordcount.values())
}

{'Midnight Picture Show': 176.0}

In [58]:
answers[
    "25"
] = f"3. {[k for k, v in corps_by_overview_wordcount.items() if v == max(corps_by_overview_wordcount.values())][0]}"
answers["25"]

'3. Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу?
по vote_average

In [59]:
data.columns.tolist()

['imdb_id',
 'budget',
 'revenue',
 'original_title',
 'cast',
 'director',
 'tagline',
 'overview',
 'runtime',
 'genres',
 'production_companies',
 'release_date',
 'vote_average',
 'release_year',
 'release_month',
 'release_day',
 'release_month_ru',
 'season',
 'cast_set',
 'genres_set',
 'production_companies_set',
 'title_lengths',
 'overview_wordcounts',
 'profit']

In [60]:

# Как здесь иногда бывает, правильные ответы могут присутствовать
# в разных вариантах ответа
# Я переформулирую задачу так: нам нужен ответ, состоящий полностью из правильных ответов

# Короче, хардкодим все предложенные фильмы во всех вариантах ответов

answer_options = [
    "Inside Out",
    "The Dark Knight",
    "12 Years a Slave",
    "BloodRayne",
    "The Adventures of Rocky & Bullwinkle",
    "Batman Begins",
    "The Lord of the Rings: The Return of the King",
    "Upside Down",
    "300",
    "Lucky Number Slevin",
    "Kill Bill: Vol. 1",
    "Upside Down",
    "Inside Out",
    "Iron Man ",
]

In [61]:
# топ 1% фильмы

top_rated_films = (
    data.sort_values(["vote_average"])
        .original_title.tail(int(data.shape[0] / 100))
        .tolist()
)

In [62]:
top_rated_films

['The Theory of Everything',
 'The Lord of the Rings: The Two Towers',
 'There Will Be Blood',
 'Mr. Nobody',
 '12 Years a Slave',
 'The Pianist',
 'Gone Girl',
 'Memento',
 'Guardians of the Galaxy',
 'Inception',
 'The Wolf of Wall Street',
 'The Grand Budapest Hotel',
 'The Lord of the Rings: The Return of the King',
 'Room',
 'Interstellar',
 'Inside Out',
 'The Imitation Game',
 'The Dark Knight']

In [63]:
# в топе берём лишь те ответы, которые есть в вариантах ответов
preanswer = sorted([f for f in top_rated_films if f in answer_options], reverse=True)[1:]

# ответ специфическим образом отформатирован, сортируем нужным образом
answers["26"] = "1. " + ", ".join(sorted(preanswer[:2]) + preanswer[2:])
answers["26"]

'1. Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


Это задание мне показалось не столь очевидно решаемым, как все предыдущие, поэтому подробно прокомментирую свой ход мыслей.

Для начала **соберём все возможные пары актёров** в множество -- позже будем искать включения множеств.

Их много, так что **законсервируем результат с помощью pickle**

In [64]:
actor_pairs_filepath = "27_actor_pairs.pkl"
actor_pairs = {}

if os.path.exists(actor_pairs_filepath):
    with open(actor_pairs_filepath, "rb") as pin:
        actor_pairs = pickle.load(pin)
else:
    for a in tqdm(actors):
        for b in actors:
            if a != b:
                actor_pairs[" & ".join(sorted([a, b]))] = {a, b}
    with open(actor_pairs_filepath, "wb") as pout:
        pickle.dump(actor_pairs, pout)

len(actor_pairs)

5798715

Теперь для **каждой пары посчитаем количество строк в файле**, где сия пара фигурирует. Решаем через множества с помощью apply. Это быстрее приблизительно в 9 раз, чем делать то же самое через циклы и строки.

Опять получается времязатратная операция.
Поскольку результат мы сохраняем в словарь, то в данном случае **используем json для сохранения нашей переменной** на диске. Текстовый формат как-то универсальнее, да и хочется попробовать разные варианты.

In [65]:
%%time


def calculate_rows_for_pairs(pairs: dict) -> dict:
    films_played_together = {}

    for key, value in tqdm(pairs.items()):
        condition = data.cast_set.apply(lambda cast: value.issubset(cast))
        if condition.sum() > 0:
            films_played_together[key] = data.imdb_id.loc[condition].shape[0]

    return films_played_together


films_together_filepath = "27_films_together.json"
if os.path.exists(films_together_filepath):
    with open(films_together_filepath, "r") as jin:
        films_together = json.loads(jin.read())
else:
    films_together = calculate_rows_for_pairs(actor_pairs)
    with open(films_together_filepath, "w") as jout:
        jout.write(json.dumps(films_together))

CPU times: user 4.45 ms, sys: 6.42 ms, total: 10.9 ms
Wall time: 19.7 ms


Отфильтруем только **значенния, равные маскимальному** в нашем словаре, и посмотрим что там у нас получается.

In [66]:
%%time
{k: v for k, v in films_together.items() if v == max(films_together.values())}

CPU times: user 5.78 s, sys: 2.78 ms, total: 5.79 s
Wall time: 5.79 s


{'Emma Watson & Rupert Grint': 8,
 'Daniel Radcliffe & Rupert Grint': 8,
 'Daniel Radcliffe & Emma Watson': 8}

**Ага! Их у этих максимальных несколько, и они равны друг другу.**

Это значит, что теперь надо выбрать то, что соответствует какому-нибудь из предложенных вариантов ответа.

Перебираем предложенные варианты и ищем пересечение с моим топом.

In [67]:
possible_answers = [
    "Johnny Depp & Helena Bonham Carter",
    "Ben Stiller & Owen Wilson",
    "Vin Diesel & Paul Walker",
    "Adam Sandler & Kevin James",
    "Daniel Radcliffe & Rupert Grint",
]

my_variants = [
    k for k, v in films_together.items() if v == max(films_together.values())
]

for index, answer_pair in enumerate(possible_answers):
    for my_pair in my_variants:
        answer_name1, answer_name2 = answer_pair.split(" & ")
        if answer_name1 in my_pair and answer_name2 in my_pair:
            answers["27"] = f"{index + 1}. {answer_pair}"

print(answers["27"])
# +

5. Daniel Radcliffe & Rupert Grint


Да, совпадает один конкретный вариант. И да, он правильный.

Я понимаю, нельзя использовать варианты ответов в коде решения, но система не даёт ввести все три правильных варианта, поэтому из трёх правильных я выбрал тот самый один, что удовлетворит систему.

ВАРИАНТ 2

In [68]:
# лучше не запускать -- потребует около 9 часов (на моей машине).
# Просто как вариант альтернативного решения

# actor_pairs = {}
#
# for a in tqdm(actors):
#     for b in actors:
#         if a != b:
#             pair = " & ".join(sorted([a, b]))
#             actor_pairs[pair] = data.loc[
#                 (data.cast.str.contains(a)) & (data.cast.str.contains(b))
#                 ].shape[0]
#
# {k: v for k, v in actor_pairs.items() if v == max(actor_pairs.values())}

# Submission

In [69]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [70]:
# и убедиться что ничего не пропустил)
len(answers)

27